In [ ]:
import numpy as np
import pandas as pd 
import torch 
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import optuna
from optuna.pruners import MedianPruner
import torch.optim as optim
import os
#import hdf5plugin
from sklearn.preprocessing import StandardScaler

In [ ]:
torch.manual_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# **Data Preprocessing**


In [ ]:
file_path = '/kaggle/input/lhc-olympics-2020-ad-r-and-d/events_anomalydetection_v2.features.h5'
df = pd.read_hdf(file_path)
print(df.shape)
print("Memory in GB:",sum(df.memory_usage(deep=True)) / (1024**3))

In [ ]:
#Separating Background and Signal Data
bg_data = df[df["label"] == 0].drop(columns = "label")
signal_data = df[df["label"] == 1].drop(columns = "label")
test_data = df.drop(columns = "label")

In [ ]:
scaler = StandardScaler()
scaler.fit(bg_data)
bg_data_scaled = scaler.transform(bg_data)
signal_data_scaled = scaler.transform(signal_data)
test_data_scaled = scaler.transform(test_data)

In [ ]:
class FeatureDataset(Dataset):
    def __init__(self,features):
        self.features = torch.tensor(features,dtype = torch.float32)
        
    def __len__(self):
        return len(self.features)

    def __getitems__(self,idx):
        x = self.features[idx]
        return (x,x)

In [ ]:
train_dataset = FeatureDataset(bg_data_scaled)
train_dataset, val_dataset = random_split(train_dataset,[0.8,0.2])
test_dataset = FeatureDataset(test_data_scaled)

# ***Linear Autoencoder***

## Hyperparameter Tuning

In [ ]:
class MyAE(nn.Module):
    def __init__(self,num_hidden_layers,input_dim,latent_dim,trial):
        super().__init__()
        encoder_layers = []
        current_dim = input_dim
        for i in range(num_hidden_layers):
            neurons_per_layer = trial.suggest_int(f"layer_{i+1}",8,128)
            encoder_layers.append(nn.Linear(current_dim,neurons_per_layer))
            encoder_layers.append(nn.ReLU())
            current_dim = neurons_per_layer
        encoder_layers.append(nn.Linear(current_dim,latent_dim))
        self.encoder = nn.Sequential(*encoder_layers)

        decoder_layers = []
        encoder_children = list(self.encoder.children())
        for layer in reversed(encoder_children):
            if isinstance(layer, nn.Linear):
                decoder_layers.append(nn.Linear(layer.out_features,layer.in_features))
                decoder_layers.append(nn.ReLU())
        decoder_layers.pop()
        decoder_layers.append(nn.Sigmoid())
        self.decoder = nn.Sequential(*decoder_layers)


    def forward(self,x):
        encoded_data = self.encoder(x)
        reconstructed_data = self.decoder(encoded_data)
        return reconstructed_data

In [ ]:
def objective(trial):
    num_hidden_layers = trial.suggest_int("num_hidden_layers",4,8)
    batch_size = trial.suggest_categorical("batch_size",[64,128,256,512])
    latent_dim = trial.suggest_int("latent_dim",2,8,step=2)
    learning_rate = trial.suggest_float("learning_rate",5e-5,1e-2,log=True)
    weight_decay = trial.suggest_float("weight_decay",1e-5,1e-3,log=True)
    epochs = trial.suggest_int("epochs",10,40,step = 5)
    optimizer_name = trial.suggest_categorical("optimizer_name",["Adam","SGD","RMSprop"])

    train_loader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True, pin_memory = True)
    test_loader = DataLoader(test_dataset,batch_size = batch_size, shuffle = False, pin_memory = True)
    val_loader = DataLoader(val_dataset,batch_size = batch_size, shuffle = False, pin_memory = True)

    input_dim = 14
    model = MyAE(num_hidden_layers,input_dim,latent_dim,trial)
    model.to(device)
    criterion = nn.MSELoss()
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay = weight_decay)
    elif optimizer_name == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay = weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay = weight_decay)

    #Training Loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            reconstructed_features = model(inputs)
            loss = criterion(reconstructed_features, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss/len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} : Loss - {epoch_loss:.4f}")


        #Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                reconstructed_features= model(inputs)
                loss = criterion(reconstructed_features, targets)
                val_loss += loss.item()
        avg_loss = val_loss/len(val_loader)
    
        trial.report(avg_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    return avg_loss

In [ ]:
pruner = MedianPruner()
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 30)

In [ ]:
print(study.best_value)
print(study.best_params)

## Model Testing

In [ ]:
#Hyper params
batch_size = 128
learning_rate = 0.0006609275044268213   
weight_decay = 2.1608381475166164e-05   
epochs = 30

In [ ]:
class AE(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(14,112),
            nn.ReLU(),
            nn.Linear(112,125),
            nn.ReLU(),
            nn.Linear(125,39),
            nn.ReLU(),
            nn.Linear(39,25),
            nn.ReLU(),
            nn.Linear(25,8),
        )

        self.decoder = nn.Sequential(
            nn.Linear(8,25),
            nn.ReLU(),
            nn.Linear(25,39),
            nn.ReLU(),
            nn.Linear(39,125),
            nn.ReLU(),
            nn.Linear(125,112),
            nn.ReLU(),
            nn.Linear(112,14),
        )

    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

test_model = AE()
test_model.to(device)
optimizer = optim.AdamW(test_model.parameters(), lr=learning_rate, weight_decay = weight_decay)

In [ ]:
#Training Loop
criterion = nn.MSELoss()
train_loader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True, pin_memory = True)
for epoch in range(epochs):
    test_model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        reconstructed_features = test_model(inputs)
        loss = criterion(reconstructed_features, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss/len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} : Loss - {epoch_loss:.4f}")

In [ ]:
#Validation loop
criterion = nn.MSELoss(reduction = "none")
val_loader = DataLoader(val_dataset,batch_size = batch_size, shuffle = False, pin_memory = True)
test_model.eval()
val_loss = []
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        reconstructed_features= test_model(inputs)
        loss = criterion(reconstructed_features, targets)
        per_event_loss = torch.mean(loss, dim=1)
        val_loss.extend(per_event_loss.cpu().numpy().tolist())
avg_loss = sum(val_loss)/len(val_dataset)
max_loss = max(val_loss)
print(f"max loss is {max_loss}")
print(f"avg loss is {avg_loss}")

In [ ]:
#Test loop
criterion = nn.MSELoss(reduction = "none")
test_loader = DataLoader(test_dataset,batch_size = batch_size, shuffle = False, pin_memory = True)
test_model.eval()
test_loss = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        reconstructed_features= test_model(inputs)
        loss = criterion(reconstructed_features, targets)
        per_event_loss = torch.mean(loss, dim=1)
        test_loss.extend(per_event_loss.cpu().numpy().tolist())

test_loss = np.array(test_loss)
all_labels = df["label"].to_numpy().astype(int)

In [ ]:
test_loss

## ROC Curve (Threshold analysis)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(all_labels, test_loss)
auc_score = roc_auc_score(all_labels, test_loss)

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
# Plot the optimal point
plt.plot(fpr[optimal_idx], tpr[optimal_idx], 'ro', label=f'Optimal Threshold = {optimal_threshold:.4f}')
plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

print(f"Optimal threshold based on ROC curve: {optimal_threshold:.4f}")

In [ ]:
#Anomaly Prediction
preds = (test_loss > optimal_threshold).astype(int)
correct_pred = (preds == all_labels)
accuracy = correct_pred.sum()/len(all_labels)
print(accuracy)
print(preds)
print(all_labels)

# ***Variational Autoencoder***

## Hyperparam Tuning

In [ ]:
#Model Architecture
class MyVAE(nn.Module):
    def __init__(self,num_hidden_layers,input_dim,latent_dim,trial):
        super().__init__()
        encoder_layers = []
        current_dim = input_dim
        for i in range(num_hidden_layers):
            neurons_per_layer = trial.suggest_int(f"layer_{i+1}",8,128)
            encoder_layers.append(nn.Linear(current_dim,neurons_per_layer))
            encoder_layers.append(nn.ReLU())
            current_dim = neurons_per_layer
        self.encoder = nn.Sequential(*encoder_layers)

        self.fc_mu = nn.Linear(current_dim, latent_dim)
        self.fc_logvar = nn.Linear(current_dim, latent_dim)

        decoder_layers = []
        decoder_layers.append(nn.Linear(latent_dim, current_dim))
        encoder_children = list(self.encoder.children())
        for layer in reversed(encoder_children):
            if isinstance(layer, nn.Linear):
                decoder_layers.append(nn.Linear(layer.out_features,layer.in_features))
                decoder_layers.append(nn.ReLU())
        decoder_layers.pop()
        self.decoder = nn.Sequential(*decoder_layers)

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 14))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_function(recon_x, x, mu, logvar, beta=1.0):
    recon_loss = nn.functional.mse_loss(recon_x, x.view(-1, 14), reduction='sum')/x.shape[0]
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim = 1).mean()
    return recon_loss + (beta * kl_div)

In [ ]:
def objective(trial):
    num_hidden_layers = trial.suggest_int("num_hidden_layers",4,8)
    batch_size = trial.suggest_categorical("batch_size",[64,128,256,512])
    latent_dim = trial.suggest_int("latent_dim",2,8,step=2)
    learning_rate = trial.suggest_float("learning_rate",5e-5,5e-3,log=True)
    weight_decay = trial.suggest_float("weight_decay",1e-5,1e-3,log=True)
    epochs = trial.suggest_int("epochs",10,40,step = 5)
    beta = trial.suggest_float("beta",0.1,1.0,log=True)
    optimizer_name = trial.suggest_categorical("optimizer_name",["AdamW","Adam"])

    train_loader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True, pin_memory = True)
    test_loader = DataLoader(test_dataset,batch_size = batch_size, shuffle = False, pin_memory = True)
    val_loader = DataLoader(val_dataset,batch_size = batch_size, shuffle = False, pin_memory = True)

    input_dim = 14
    model = MyVAE(num_hidden_layers,input_dim,latent_dim,trial)
    model.to(device)
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay = weight_decay)
    elif optimizer_name == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay = weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay = weight_decay)

    #Training Loop
    for epoch in range(epochs):
        model.train()
        running_total_loss = 0.0
        running_recon_loss = 0.0
        running_kld_loss = 0.0
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            reconstructed_features, mu, logvar = model(inputs)
            recon_loss = nn.functional.mse_loss(reconstructed_features, targets, reduction='sum')
            kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            total_loss = recon_loss + (beta * kl_div)
            #loss = loss_function(reconstructed_features, targets, mu, logvar)
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            running_total_loss += total_loss.item()
            running_recon_loss += recon_loss.item()
            running_kld_loss += kl_div.item()
        avg_total_loss = running_total_loss / len(train_loader.dataset)
        avg_recon_loss = running_recon_loss / len(train_loader.dataset)
        avg_kld_loss = running_kld_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} | "
          f"Total Loss: {avg_total_loss:.4f} | "
          f"Recon Loss: {avg_recon_loss:.4f} | "
          f"KLD: {avg_kld_loss:.4f}")


        #Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                reconstructed_features, mu, logvar = model(inputs)
                loss = loss_function(reconstructed_features, targets, mu, logvar,beta)
                val_loss += loss.item()
        avg_loss = val_loss/len(val_loader)
    
        trial.report(avg_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    return avg_loss

In [ ]:
pruner = MedianPruner()
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 50)

In [ ]:
print(study.best_value)
print(study.best_params)

## Model Testing

In [ ]:
#Hyperparams
beta = 0.14134371590127678
batch_size = 256
learning_rate = 0.003290904745561437
weight_decay = 1.4399478254951877e-05

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(14,44),
            nn.ReLU(),
            nn.Linear(44, 127),
            nn.ReLU(),
            nn.Linear(127, 60),
            nn.ReLU(),
            nn.Linear(60, 97),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(97, 8) # for mean
        self.fc_logvar = nn.Linear(97, 8) # for log variance

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(8, 97),
            nn.ReLU(),
            nn.Linear(97,60),
            nn.ReLU(),
            nn.Linear(60, 127),
            nn.ReLU(),
            nn.Linear(127, 44),
            nn.ReLU(),
            nn.Linear(44, 14)
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 14))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def loss_function(recon_x, x, mu, logvar, beta=1.0):
    recon_loss = nn.functional.mse_loss(recon_x, x.view(-1, 14), reduction='sum')
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + (beta * kl_div)

In [ ]:
input_dim = 14
vae_model = VAE()
vae_model.to(device)
optimizer = optim.AdamW(vae_model.parameters(), lr=learning_rate, weight_decay = weight_decay)

In [ ]:
#Training Loop
model_save_path = 'best_model.pth'
patience = 10 
best_val_loss = float('inf')
patience_counter = 0
anneal_epochs = 10
epochs = 50

train_loader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True, pin_memory = True)
val_loader = DataLoader(val_dataset,batch_size = batch_size, shuffle = False, pin_memory = True)

for epoch in range(epochs):
    print(f"--- Epoch {epoch+1}/{epochs} ---")
    vae_model.train()
    running_total_loss = 0.0
    running_recon_loss = 0.0
    running_kld_loss = 0.0
    kl_weight = min(1.0, epoch/anneal_epochs) * beta    
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        reconstructed_features, mu, logvar = vae_model(inputs)
        recon_loss = nn.functional.mse_loss(reconstructed_features, targets, reduction='sum')
        kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        total_loss = recon_loss + (kl_weight * kl_div)
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        running_total_loss += total_loss.item()
        running_recon_loss += recon_loss.item()
        running_kld_loss += kl_div.item()
    avg_total_loss = running_total_loss / len(train_loader.dataset)
    avg_recon_loss = running_recon_loss / len(train_loader.dataset)
    avg_kld_loss = running_kld_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs} | "
      f"Total Loss: {avg_total_loss:.4f} | "
      f"Recon Loss: {avg_recon_loss:.4f} | "
      f"KLD: {avg_kld_loss:.4f}")

     # --- Validation Phase ---
    vae_model.eval()
    current_val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            reconstructed_features, mu, logvar = vae_model(inputs)
            loss = loss_function(reconstructed_features, targets, mu, logvar,beta=beta)
            current_val_loss += loss.item()    
    avg_val_loss = current_val_loss / len(val_loader.dataset)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    # --- Early Stopping Logic ---
    if avg_val_loss < best_val_loss:
        print(f"Validation loss improved from {best_val_loss:.4f} to {avg_val_loss:.4f}. Saving model...")
        best_val_loss = avg_val_loss
        torch.save(vae_model.state_dict(), model_save_path)
        patience_counter = 0 # Reset patience
    else:
        patience_counter += 1
        print(f"No improvement in validation loss for {patience_counter} epoch(s).")
        if patience_counter >= patience:
            print(f"Early stopping triggered after {patience} epochs with no improvement.")
            break

print("\n--- Training finished ---")

In [ ]:
#Test Loop
from sklearn.metrics import roc_auc_score
vae_model = VAE()
vae_model.to(device)
vae_model.load_state_dict(torch.load('best_model.pth'))
vae_model.eval()
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
all_recon_errors = []
all_klds = []
all_total_losses = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        reconstructed_features, mu, logvar = vae_model(inputs)
        recon_error_per_event = torch.sum(nn.functional.mse_loss(reconstructed_features, targets, reduction='none'), dim=1)
        kld_per_event = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)
        total_loss_per_event = recon_error_per_event + (beta * kld_per_event)

        all_recon_errors.extend(recon_error_per_event.cpu().numpy().tolist())
        all_klds.extend(kld_per_event.cpu().numpy().tolist())
        all_total_losses.extend(total_loss_per_event.cpu().numpy().tolist())
        
all_labels = df["label"].to_numpy().astype(int)
all_recon_errors = np.array(all_recon_errors)
all_klds = np.array(all_klds)
all_total_losses = np.array(all_total_losses)

auc_recon = roc_auc_score(all_labels, all_recon_errors)
auc_kld = roc_auc_score(all_labels, all_klds)
auc_total = roc_auc_score(all_labels, all_total_losses)

print(f"AUC using Reconstruction Error: {auc_recon:.4f}")
print(f"AUC using KL Divergence:        {auc_kld:.4f}")
print(f"AUC using Total VAE Loss:       {auc_total:.4f}")

## ROC Curve (Threshold Analysis)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(all_labels, all_total_losses)
auc_score = auc_total

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
# Plot the optimal point
plt.plot(fpr[optimal_idx], tpr[optimal_idx], 'ro', label=f'Optimal Threshold = {optimal_threshold:.4f}')
plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

print(f"Optimal threshold based on ROC curve: {optimal_threshold:.4f}")

In [ ]:
#Anomaly Prediction
preds = (all_total_losses > optimal_threshold).astype(int)
correct_pred = (preds == all_labels)
accuracy = correct_pred.sum()/len(all_labels)
print(accuracy)
print(preds)
print(all_labels)